In [1]:
%config Completer.use_jedi=False
%load_ext autoreload
%autoreload 2

In [4]:
from pytube import Channel, YouTube

import whisper

In [39]:
whisper??

Type:        module
String form: <module 'whisper' from '/Users/ahmedbesbes/opt/anaconda3/lib/python3.9/site-packages/whisper/__init__.py'>
File:        ~/opt/anaconda3/lib/python3.9/site-packages/whisper/__init__.py
Source:     
import hashlib
import io
import os
import urllib
import warnings
from typing import List, Optional, Union

import torch
from tqdm import tqdm

from .audio import load_audio, log_mel_spectrogram, pad_or_trim
from .decoding import DecodingOptions, DecodingResult, decode, detect_language
from .model import Whisper, ModelDimensions
from .transcribe import transcribe


_MODELS = {
    "tiny.en": "https://openaipublic.azureedge.net/main/whisper/models/d3dd57d32accea0b295c96e26691aa14d8822fac7d9d27d5dc00b4ca2826dd03/tiny.en.pt",
    "tiny": "https://openaipublic.azureedge.net/main/whisper/models/65147644a518d12f04e32d6f3b26facc3f8dd46e5390956a9424a650c0ce22b9/tiny.pt",
    "base.en": "https://openaipublic.azureedge.net/main/whisper/models/25a8566e1d0c1e2231d1c762132c

In [23]:
whisper_model = whisper.load_model("base")

In [43]:
whisper.load_model

<function whisper.load_model(name: str, device: Union[str, torch.device, NoneType] = None, download_root: str = None, in_memory: bool = False) -> whisper.model.Whisper>

In [17]:
url = "https://youtu.be/_3sUkXUYH-Y"

In [18]:
youtube = YouTube(url)

In [19]:
%%time
path = youtube.streams.filter(only_audio=True)[0].download(filename="audio.mp4")

CPU times: user 2.22 s, sys: 41 ms, total: 2.27 s
Wall time: 3.14 s


In [40]:
path

'/Users/ahmedbesbes/Documents/perso/whisper/notebooks/audio.mp4'

In [24]:
%%time
transcription = whisper_model.transcribe(path)

/Users/ahmedbesbes/opt/anaconda3/lib/python3.9/site-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


CPU times: user 51.6 s, sys: 4.89 s, total: 56.5 s
Wall time: 7.58 s


In [25]:
transcription

{'text': " Does Little Ross like dinosaurs by any chance? Yeah, there. Molly talks about why. How would he like to come with me to the museum of natural history after everyone else has left? Just the two of us. And he can touch anything he wants. I just heard it. How you must have heard it. And that's not good. Let me start again. I'm a paleontologist. You'll be there with us. And the touching refers only to bones, fossils. You could really arrange that? You let Rachel come back, and it's done. Well, I guess having Rachel back wouldn't be the worst thing in the world. Yeah, yeah, stop. Thank you. This is great. Thank you so much. And I swear your kid's going up the time of his life. That's great. And worry about Little Ross. He's always reading. He's collecting rocks, and he's obsessed with dinosaurs. He'll be fine.",
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 3.0,
   'text': ' Does Little Ross like dinosaurs by any chance?',
   'tokens': [4402, 8022, 16140, 411, 

### Keyword extraction with Keybert

In [26]:
from keybert import KeyBERT

In [27]:
doc = """
         Supervised learning is the machine learning task of learning a function that
         maps an input to an output based on example input-output pairs. It infers a
         function from labeled training data consisting of a set of training examples.
         In supervised learning, each example is a pair consisting of an input object
         (typically a vector) and a desired output value (also called the supervisory signal).
         A supervised learning algorithm analyzes the training data and produces an inferred function,
         which can be used for mapping new examples. An optimal scenario will allow for the
         algorithm to correctly determine the class labels for unseen instances. This requires
         the learning algorithm to generalize from the training data to unseen situations in a
         'reasonable' way (see inductive bias).
      """

In [34]:
transcript = transcription["text"]

In [42]:
transcription

{'text': " Does Little Ross like dinosaurs by any chance? Yeah, there. Molly talks about why. How would he like to come with me to the museum of natural history after everyone else has left? Just the two of us. And he can touch anything he wants. I just heard it. How you must have heard it. And that's not good. Let me start again. I'm a paleontologist. You'll be there with us. And the touching refers only to bones, fossils. You could really arrange that? You let Rachel come back, and it's done. Well, I guess having Rachel back wouldn't be the worst thing in the world. Yeah, yeah, stop. Thank you. This is great. Thank you so much. And I swear your kid's going up the time of his life. That's great. And worry about Little Ross. He's always reading. He's collecting rocks, and he's obsessed with dinosaurs. He'll be fine.",
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 3.0,
   'text': ' Does Little Ross like dinosaurs by any chance?',
   'tokens': [4402, 8022, 16140, 411, 

In [37]:
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(
    transcript, keyphrase_ngram_range=(1, 2), stop_words=["ross"]
)